# Análisis de Dosis por Procedimiento y Proyección
Este notebook analiza los datos de dosis de radiación registrados en un archivo Excel. Se organiza la información por procedimiento y proyección, y se visualizan los resultados mediante gráficos tipo candlestick para facilitar la interpretación de los valores de dosis y dosis total.

## 1. Carga de datos
En esta sección se importan las librerías necesarias y se carga el archivo Excel con los datos de dosis.

In [9]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


df = pd.read_excel('procedimientos_demo.xlsx')


#change the name for the column procedimiento to procedimiento_id and the procedimeinto_nombre to procedimiento_nombre
df = df.rename(columns={'procedimiento': 'procedimiento_id', 'procedimiento_nombre': 'procedimiento'})
df.head()

,procedimiento_id,procedimiento,proyeccion,dosis,dosis total,fecha,paciente_id,comentario
0,EST-00001,Histerosalpingografía (HSG),AP,0.72,5.63,2025-05-18,PX-00001,NaN
1,EST-00001,Histerosalpingografía (HSG),PA,4.53,NaN,2025-05-18,PX-00001,NaN
2,EST-00001,Histerosalpingografía (HSG),LAT,0.38,NaN,2025-05-18,PX-00001,NaN
3,EST-00002,Uretrocistografía miccional (VCUG),LAT,0.85,7.31,2025-02-11,PX-00002,NaN
4,EST-00002,Uretrocistografía miccional (VCUG),AP,2.89,NaN,2025-02-11,PX-00002,NaN


## 2. Visualización de los datos
Se generan gráficos tipo candlestick para visualizar la distribución de dosis por procedimiento y proyección, así como la dosis total por procedimiento.

### 2.1) Sin usar un ID y solo por procedimiento

In [10]:
# --- 1) Nos quedamos solo con las filas donde 'dosis total' no es NaN ---
df_totales = df.dropna(subset=["dosis total"])

# --- 2) (Opcional) Ordenar procedimientos por mediana para que el eje X quede más legible ---
orden = (df_totales
         .groupby("procedimiento")["dosis total"]
         .median()
         .sort_values()
         .index
         .tolist())

# --- 3) Graficar boxplots por procedimiento ---
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Box(
    x=df_totales["procedimiento"],      # categorías (un box por procedimiento)
    y=df_totales["dosis total"],        # valores
    name="Dosis total",
    boxpoints="outliers",               # muestra solo outliers (usa "all" para todos los puntos)
    boxmean=True                        # dibuja la media (usa "sd" para mostrar también ±1 SD)
))

fig.update_layout(
    title="Distribución de 'Dosis total' por procedimiento (boxplots)",
    xaxis_title="Procedimiento",
    yaxis_title="Dosis total",
)

# aplica el orden por mediana en el eje X (quita esta línea si no quieres reordenar)
fig.update_xaxes(categoryorder="array", categoryarray=orden)

fig.show()


#### Candlesticks

In [11]:


# --- 1) Nos quedamos solo con las filas donde 'dosis total' no es NaN ---
df_totales = df.dropna(subset=["dosis total"])

# --- 2) Calcular estadísticos por procedimiento ---
def q25(x): return np.nanpercentile(x, 25)
def q75(x): return np.nanpercentile(x, 75)

stats = (df_totales
         .groupby("procedimiento")   # <--- ahora agrupamos por el nombre
         .agg(
             low=("dosis total", "min"),
             q1=("dosis total", q25),
             median=("dosis total", "median"),
             q3=("dosis total", q75),
             high=("dosis total", "max"),
             n=("dosis total", "count")
         )
         .reset_index())

# --- 3) Graficar un solo candlestick con los 5 procedimientos ---
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=stats["procedimiento"],
    open=stats["q1"],    # límite inferior de la caja (IQR)
    high=stats["high"],  # mecha superior
    low=stats["low"],    # mecha inferior
    close=stats["q3"],   # límite superior de la caja (IQR)
    name="Dosis total"
))

# Mediana como punto 
fig.add_trace(go.Scatter(
    x=stats["procedimiento"],
    y=stats["median"],
    mode="markers",
    name="Mediana",
    marker=dict(size=8, color="black")
))

fig.update_layout(
    title="Distribución de 'Dosis total' por procedimiento",
    xaxis_title="Procedimiento",
    yaxis_title="Dosis total",
    xaxis_rangeslider_visible=False
)

fig.show()


### 2.2) Seccionando con ID para mayor seguridad y solo por procedimiento

In [12]:
# --- Boxplots en lugar de candlesticks para 'dosis total' (1 valor por estudio) ---


# --- 1) Quedarnos con un valor de "dosis total" por estudio (procedimiento_id) ---
# Si existe el nombre del procedimiento, lo conservamos; si no, usamos la propia ID como nombre.
if "procedimiento" in df.columns:
    total_por_estudio = (df
        .dropna(subset=["dosis total"])
        .sort_values("procedimiento_id")
        .groupby("procedimiento_id", as_index=False)
        .agg({
            "dosis total": "max",   # o "first" si prefieres
            "procedimiento": "first"
        })
    )
else:
    total_por_estudio = (df
        .dropna(subset=["dosis total"])
        .sort_values("procedimiento_id")
        .groupby("procedimiento_id", as_index=False)["dosis total"].max()
    )
    # Sin columna de nombre, usamos la ID como etiqueta (recomendado unir un catálogo si lo tienes)
    total_por_estudio["procedimiento"] = total_por_estudio["procedimiento_id"].astype(str)

# --- 2) (Opcional) Mantener sólo 5 procedimientos objetivo y ordenarlos ---
objetivo = [
    "Histerosalpingografía (HSG)",
    "Esofagograma",
    "Tránsito intestinal",
    "Enema opaco",
    "Uretrocistografía miccional (VCUG)",
]
presentes = [p for p in objetivo if p in set(total_por_estudio["procedimiento"])]

# Si los 5 (o algunos) están presentes, filtramos a esos; si no, graficamos todos.
df_plot = (total_por_estudio
           if not presentes
           else total_por_estudio[total_por_estudio["procedimiento"].isin(presentes)].copy())

# Orden del eje X: fijo (si filtramos a los 5) o por mediana ascendente
if presentes:
    orden_x = presentes
else:
    orden_x = (df_plot.groupby("procedimiento")["dosis total"]
               .median()
               .sort_values()
               .index
               .tolist())

# --- 3) Boxplots por procedimiento ---
fig = go.Figure()

fig.add_trace(go.Box(
    x=df_plot["procedimiento"],       # un box por procedimiento
    y=df_plot["dosis total"],         # valores (1 por estudio)
    name="Dosis total por estudio",
    boxpoints="outliers",             # muestra outliers (usa "all" para todos los puntos)
    boxmean=True                      # marca la media (usa "sd" para mostrar también ±1 SD)
))

fig.update_layout(
    title="Distribución de 'Dosis total' por procedimiento (boxplots, 1 valor por estudio)",
    xaxis_title="Procedimiento",
    yaxis_title="Dosis total",
)

fig.update_xaxes(categoryorder="array", categoryarray=orden_x)

fig.show()


## 3) Para un procedimiento espesifico:

In [13]:
# --- 1) Filtrar al procedimiento de interés ---
proc_name = "Esofagograma"
df_proc = df[df["procedimiento"] == proc_name].dropna(subset=["dosis"])

# --- 2) (Opcional) ordenar proyecciones por mediana para un eje X más legible ---
orden = (df_proc.groupby("proyeccion")["dosis"]
         .median()
         .sort_values()
         .index
         .tolist())

# --- 3) Graficar boxplots por proyección ---
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Box(
    x=df_proc["proyeccion"],   # un box por proyección
    y=df_proc["dosis"],        # valores de dosis
    name=f"Dosis en {proc_name}",
    boxpoints="outliers",      # muestra outliers (usa "all" para todos los puntos)
    boxmean=True               # muestra la media (además de la mediana por defecto)
))

fig.update_layout(
    title=f"Distribución de dosis por proyección ({proc_name})",
    xaxis_title="Proyección",
    yaxis_title="Dosis",
)

# Aplica el orden elegido al eje X
fig.update_xaxes(categoryorder="array", categoryarray=orden)

fig.show()


In [14]:
# --- 1) Filtrar al procedimiento de interés ---
proc_name = "Esofagograma"
df_proc = df[df["procedimiento"] == proc_name]

# --- 2) Calcular estadísticos por proyección ---
def q25(x): return np.nanpercentile(x, 25)
def q75(x): return np.nanpercentile(x, 75)

stats_proj = (df_proc.groupby("proyeccion")
              .agg(low=("dosis", "min"),
                   q1=("dosis", q25),
                   median=("dosis", "median"),
                   q3=("dosis", q75),
                   high=("dosis", "max"),
                   n=("dosis", "count"))
              .reset_index())

# --- 3) Graficar candlestick por proyección ---
fig = go.Figure()

fig.add_trace(go.Candlestick(
    x=stats_proj["proyeccion"],
    open=stats_proj["q1"],     # límite inferior de la caja (IQR)
    high=stats_proj["high"],   # mecha superior
    low=stats_proj["low"],     # mecha inferior
    close=stats_proj["q3"],    # límite superior de la caja (IQR)
    name=f"Dosis en {proc_name}"
))

# Agregar la mediana como marcador
fig.add_trace(go.Scatter(
    x=stats_proj["proyeccion"],
    y=stats_proj["median"],
    mode="markers",
    name="Mediana",
    marker=dict(size=8, color="black")
))

fig.update_layout(
    title=f"Distribución de dosis por proyección ({proc_name})",
    xaxis_title="Proyección",
    yaxis_title="Dosis",
    xaxis_rangeslider_visible=False
)

fig.show()